In [1]:
import gym
import custom_gym
import numpy as np
import torch

env = gym.make('CustomPendulum-v0')
s_dim = env.reset().shape[0]
a_dim = env.action_space.sample().shape[0]
print(s_dim,a_dim)

3 1


C:\Users\toru.hishinuma\anaconda3\envs\note\lib\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
offline_data = np.load('np_offline_data.npy')
offline_data = torch.from_numpy(offline_data.astype(np.float32)).clone()
print(offline_data.shape)

torch.Size([100, 200, 8])


In [3]:
def datapoint2sads(datapoint):
    s = datapoint[: s_dim]
    a = datapoint[s_dim : (s_dim+a_dim)]
    #ds = datapoint[(s_dim+a_dim) : (2*s_dim+a_dim)] - s
    ds = datapoint[(s_dim+a_dim) : (2*s_dim+a_dim)]
    return s,a,ds

def gaussian_likelihood_loss(y, mu, logvar):
    # 分散行列Varが対角成分var_iの対角行列の場合には、log(det|Var|) = log(prod_i var_i) = sum_i log(var_i) 
    return 0.5 * torch.mean(((y-mu)**2) * torch.exp(-logvar) + logvar)


def kld(mu1, logvar1, mu2, logvar2):
    # kld(p1|p2) = E_{z~p1}[ log p1(z) - log p2(z) ]
    tmp1 = 0.5 * (logvar2 - logvar1) # log (sigma2/sigma1)
    tmp2 = 0.5*(torch.exp(logvar1)+(mu1-mu2)**2) / torch.exp(logvar2) # (sigma1^2+(mu1-mu2)^2)/(2*sigma2^2)
    return torch.mean(tmp1 + tmp2)

In [4]:
x_dim = s_dim+a_dim
y_dim = s_dim
z_dim = 1
g_dim = 1
#f_phi


In [5]:
class Encoder(torch.nn.Module):
    def __init__(self, s_dim, a_dim, z_dim, g_dim):
        super(Encoder, self).__init__()
        self.net_f = torch.nn.Sequential(
                            torch.nn.Linear(2*s_dim+a_dim, 64),
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, g_dim),
                            )
        self.net_q = torch.nn.Sequential(
                            torch.nn.Linear(g_dim, 64),
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 2*z_dim),
                            )

    def forward(self, data_m):
        g_m = self.net_f(data_m).mean().reshape(1,-1)
        return self.net_q(g_m).flatten()
        

        


In [6]:
class Decoder(torch.nn.Module):
    def __init__(self, s_dim, a_dim, z_dim):  
        super(Decoder, self).__init__()
        self.net_phat = torch.nn.Sequential(
                            torch.nn.Linear(s_dim+a_dim+z_dim, 64),
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 2*s_dim),
                            )
        
    def forward(self, saz):
        return self.net_phat(saz)
        

In [7]:
class unweightedVI(torch.nn.Module):
    def __init__(self, s_dim, a_dim, z_dim):
        super(unweightedVI, self).__init__()
        self.enc = Encoder(s_dim, a_dim, z_dim, g_dim)
        self.dec = Decoder(s_dim, a_dim, z_dim)
        self.prior = torch.nn.Parameter(torch.zeros(2*z_dim))
        self.nu = 1e-2
        

    def loss(self, offline_data_m):
        z_mu_logvar = self.enc(offline_data_m)

        # reparametrization trick
        eps = torch.randn_like(z_mu_logvar[:z_dim])
        std = torch.exp(0.5 * z_mu_logvar[z_dim:])
        z = (eps*std+z_mu_logvar[:z_dim]) * torch.ones(offline_data.shape[1],z_dim)

        saz = torch.cat([offline_data[m,:,:(s_dim+a_dim)],z],dim=1)

        ds_mu_logvar = self.dec(saz)

        ret_loss = 0

        # y = offline_data[m,:,(s_dim+a_dim):-1]
        # mu = ds_mu_logvar[:,:s_dim]
        # logvar = ds_mu_logvar[:,s_dim:]        
        ret_loss += gaussian_likelihood_loss(offline_data[m,:,(s_dim+a_dim):-1],
                                         ds_mu_logvar[:,:s_dim],
                                         ds_mu_logvar[:,s_dim:]) # approx of E_{z~q}[ - log p(y|x,z) ]
        
        ret_loss += self.nu * kld(z_mu_logvar[:z_dim], 
                              z_mu_logvar[z_dim:],
                              self.prior[:z_dim],
                              self.prior[z_dim:]) # nu * E_{z~q}[ log q(z) - log p(z) ]
        return ret_loss
    
    
vi = unweightedVI(s_dim, a_dim, z_dim)

optimizer = torch.optim.Adam(vi.parameters(),lr=0.001)

for i in range(10):
    m = np.random.randint(offline_data.shape[0])
    optimizer.zero_grad()
    loss = vi.loss(offline_data[m,:,:-1])
    loss.backward()
    optimizer.step()

        #m = np.random.randint(offline_data.shape[0])
        #z_mu_logvar = enc(offline_data[m,:,:-1])
        

